<a href="https://colab.research.google.com/github/rmngllnn/Stage2021/blob/main/POS_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import spacy
import csv
import string as str
import json

## Aperçu des données

28 834 entrées et 46 variables par entrée

In [ ]:
df = pd.read_csv("DEMOCRATIE_ET_CITOYENNETE.csv", low_memory = False)
df.shape

df.head()

ParserError: ignored

## Extraction des données

https://docs.python.org/fr/3/library/csv.htmlhttps://docs.python.org/fr/3/library/csv.html

In [ ]:
def ponctuation(texte):
    for p in str.punctuation:
        texte = texte.replace(p, " ")
    return texte

In [ ]:
#Extraction des données dans une liste de listes
#len(ex) = 28385 soit le nombre d’observation

observations = []
with open("../DEMOCRATIE_ET_CITOYENNETE.csv", newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if row != [] : observations.append(row)

In [ ]:
print(len(observations))

28385


In [ ]:
print(observations[0][0:10])

['reference', 'title', 'createdAt', 'publishedAt', 'updatedAt', 'trashed', 'trashedStatus', 'authorId', 'authorType', 'authorZipCode']


In [ ]:
del observations[0][0:10]

In [ ]:
print(observations[1])

for obs in observations[1]:
    print(obs)

['1-2', 'Les augmentations de rémunérations', '2019-01-22 09:38:41', '2019-01-22 09:38:41', '', 'false', '', 'VXNlcjo4Mjc4NzQxYS0xZTFkLTExZTktOTRkMi1mYTE2M2VlYjExZTE=', 'Citoyen / Citoyenne', '79190', 'Le citoyen', 'Non', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Afin d’éviter de creuser les inégalités ne plus augmenter les salaires, les pensions, le point d’indice des fonctionnaires, en pourcentage mais en euros. Ainsi celui qui gagne 1,000 euros percevra la même augmentation que celui qui gagne 10,000 euros. Alors que depuis des décennies l’augmentation par pourcentage avantage les plus gros revenus\xa0: 1\xa0% est égal à 10 euros pour le premier et à 100 euros pour le second. Au bout de 10 augmentations le second aura eu l’intégralité de la valeur du salaire du premier… C’est comme ça que depuis 40 ans le fossé se creuse']
1-2
Les augmentations de rémunérations
2019-01-22 09:38:41
2019-01-22 0

## POS-Tagging using SpaCy

In [ ]:
#NE PAS RUN cette cellule sauf sur Google Colab
#Permet d'installer le module sur le serveur distant utilisé.

try:
  nlp = spacy.load("fr_core_news_sm", disable=['ner'])
except:
  from spacy.cli import download
  download('fr_core_news_sm')
  nlp = spacy.load("fr_core_news_sm", disable=['ner'])

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [ ]:
nlp = spacy.load("fr_core_news_sm")
#doc = nlp(d) # /!\ Doit être de type String

doc = []
nouns = []
adj = []
pronouns = []

def pos_tagger(data):
  for obs in data:
      for sent in obs:
          doc = nlp(sent)
          for token in doc:
            if token.pos_ == "NOUN" : nouns.append((token.text, token.pos_))
            elif token.pos_ == "ADJ" : adj.append((token.text, token.pos_))
            elif token.pos_ == "PRON" : pronouns.append((token.text, token.pos_))
            

In [ ]:
pos_tagger(observations)

KeyboardInterrupt: ignored

## Sérialisation des données

Données sérialisées au format json car facilement lisible hors programmation  

https://www.codeflow.site/fr/article/python-json  

La cellule ci-dssous ne doit être runé qu'une seule fois

In [ ]:
def serialisation_data (data, title):
  """
  Serialize data in a json file
  -> Title mus be a string : title.json
  <- Save a file in desktop
  """

  with open(title, "w+") as file:
    json.dump(data, file)

'with open("nouns.json", "w+") as fichier:\n  json.dump(nouns, fichier)\n\nwith open("adjectives.json", "w+") as fichier:\n  json.dump(adj, fichier)\n \nwith open("pronouns.json", "w+") as fichier:\n  json.dump(pronouns, fichier)\n  #fichier.write(pronouns)'

In [ ]:
serialisation_data(a, "test.json")